In [20]:
import os
import pandas as pd
from PIL import Image
import torch
from torchvision import transforms
import torchvision.models as models
import json
from torch.utils.data import Dataset
import torch.nn as nn
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import torchvision

In [21]:
import numpy as np
import cv2
from PIL import Image
import torch.nn as nn

class GreyscaleContrast(nn.Module):
    """
    Convert the image to greyscale
    Apply histogram equalisation to increase contrast
    """

    def __init__(self):
        super().__init__()

    def forward(self, img):
        # Convert PIL Image to numpy array
        img = np.array(img)

        # Convert image to greyscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply histogram equalization
        img = cv2.equalizeHist(img)

        # Convert numpy array back to PIL Image
        img = Image.fromarray(img)

        return img


In [ ]:
import os
import pandas as pd

# Define transformations for test data (modify as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    GreyscaleContrast(),
    transforms.Grayscale(num_output_channels=3),
    transforms.ColorJitter(brightness=0.01),
    transforms.ToTensor(),
    # Add other transformations as needed
])

# Load the trained model
model = torchvision.models.resnet18(pretrained=False)  # Make sure to set pretrained=False
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(nn.Linear(num_ftrs, 1), nn.Sigmoid())
model.load_state_dict(torch.load("agar3000.pth", map_location=torch.device('cpu')))  # Load the saved model

# Prepare test data
test_data_dir = "test_data"  # Change this to the path of your test data directory
test_image_paths = [
    os.path.join(test_data_dir, filename)
    for filename in os.listdir(test_data_dir)
    if filename.endswith(".jpg")
]

# Perform predictions on the test data
device = torch.device("cpu")  # Use CPU for inference
model = model.to(device)
model.eval()
predictions = []
for image_path in test_image_paths:
    image_id = os.path.basename(image_path)  # Extract file ID
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)

    output = model(image)

    threshold = 0.5
    predicted = (output >= threshold).int().item()

    # Append a dictionary containing file ID and prediction to the predictions list
    predictions.append({'ID': image_id, 'TARGET': predicted})

# Now 'predictions' contains the predictions for each image in the test data

# Create a DataFrame from predictions
df = pd.DataFrame(predictions)

# Save predictions to a CSV file
df.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")